In [ ]:
!pip install planet

     |████████████████████████████████| 61kB 2.2MB/s 
  Created wheel for requests-futures: filename=requests_futures-0.9.7-cp36-none-any.whl size=5056 sha256=b969147ab4c9cb56e3b569ecb9e6555799fa1c92d98f07aec56539da7b99adb1
  Stored in directory: /root/.cache/pip/wheels/71/fb/7a/0caacf78e523a182dadd1d042792801cfd53f27c77b945b64a
Successfully built requests-futures


In [ ]:
import json
import os
import pickle
from planet.api import filters
import pathlib
import time
import numpy as np
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth
from PIL import Image
import glob
from io import BytesIO

In [ ]:
# API Key stored as an env variable
PLANET_API_KEY = '543493b065b14c97aa9b66b26d0ea6f7'
orders_url = 'https://api.planet.com/compute/ops/orders/v2'
# set up requests to work with api
auth = HTTPBasicAuth(PLANET_API_KEY, '')
headers = {'content-type': 'application/json'}
response = requests.get(orders_url, auth=auth)
from google.colab import drive
drive.mount('/content/drive')
response

Mounted at /content/drive


<Response [200]>

In [ ]:
def diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))

prev_order = ['20191025_181156_1003',
   '20191124_181109_1004',
   '20200101_181109_1010',
   '20200113_183201_20_106e',
   '20200118_181058_1005',
   '20200122_183125_43_1065',
   '20200129_164025_0f36',
   '20200218_180102_0e0f',
   '20200226_163513_0f36',
   '20200315_174404_68_2278']

In [ ]:
with open("/content/drive/My Drive/CS_229_project/img_data/file2siteGroup2lte30.txt", "rb") as fp:
  l = pickle.load(fp)

In [ ]:
site_files = {}
sites = l['site'].drop_duplicates().to_list()
for site in sites:
  files = l[l['site'] == site].file.tolist()
  site_files[site] = files
site_files.keys()

dict_keys([1016, 2004, 2002, 1201, 10, 11, 1004, 6012, 9002, 3001, 1010, 2011, 9001, 5001, 6001, 7, 1014, 1013, 2022, 4003, 9, 16, 1018, 500, 1008, 1021, 9000, 1025])

Helper functions

In [ ]:
# define helpful functions for submitting, polling, and downloading an order
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    
    if not response.ok:
        raise Exception(response.content)

    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

def poll_for_success(order_url, auth, num_loops=50):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        success_states = ['success', 'partial']
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        time.sleep(60)
        
def download_order(order_url, auth, overwrite=False):
    r = requests.get(order_url, auth=auth)
    print(r)
    response = r.json()
    results = response['_links']['results']
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    results_paths = [pathlib.Path(os.path.join('data', n)) for n in results_names]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name, path in zip(results_urls, results_names, results_paths):
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
    return dict(zip(results_names, results_paths))

# define helpful functions for visualizing downloaded imagery
def show_rgb(img_file):
    with rasterio.open(img_file) as src:
        b,g,r,n = src.read()

    rgb = np.stack((r,g,b), axis=0)
    show(rgb/rgb.max())
    
def show_gray(img_file):
    with rasterio.open(img_file) as src:
        g = src.read(1)
    show(g/g.max())

Order with Clipping

In [ ]:
# Load the clipping geometry
with open("/content/drive/My Drive/CS_229_project/areas2.geojson") as f:
    esri_output  = json.loads(f.read())
#converts esri multipolygon geojson to planet geojson
all_areas = [item['geometry']['coordinates'] for item in esri_output['features']]
# grab the siteNum for batch downloads
siteNum = [item['properties']['Site_Number'] for item in esri_output['features']]
coords = {}
i = 0
for site in siteNum:
  coords[site] = all_areas[i]
  i += 1

In [ ]:
coords.keys()

dict_keys([2022, 9, 10, 16, 500, 1010, 1016, 5001, 6001, 9000, 9001, 9002, 1008, 1201, 6012, 11, 1013, 1014, 1018, 1021, 1025, 2004, 2011, 3001, 4003, 7, 1004, 2002])

In [ ]:
site_files.keys()

dict_keys([1016, 2004, 2002, 1201, 10, 11, 1004, 6012, 9002, 3001, 1010, 2011, 9001, 5001, 6001, 7, 1014, 1013, 2022, 4003, 9, 16, 1018, 500, 1008, 1021, 9000, 1025])

In [ ]:
# optional take key out
del site_files[1016]
del site_files[2004]
del site_files[2002]
del site_files[1201]
del site_files[10]
del site_files[11]
del site_files[1004]
del site_files[6012]
del site_files[9002]
del site_files[3001]
del site_files[1010]
del site_files[2011]
del site_files[9001]
del site_files[6001]
del site_files[5001]
del site_files[7]
del site_files[1014]
del site_files[1013]
del site_files[2022]
del site_files[4003]
del site_files[9]
del site_files[16]
del site_files[1018]
del site_files[500]
del site_files[1008]


#val = site_files[9004]
#site_files.clear()
#site_files[9004] = val
site_files.keys()

dict_keys([1021, 9000, 1025])

In [ ]:
for site in site_files.keys():
  # define products part of order
  same_src_products = [
      {
        "item_ids": site_files[site],
        "item_type": "PSScene3Band",
        "product_bundle": "visual"
      }
  ]
  # Define clip area
  clip_aoi = {
      "type": "Polygon",
      "coordinates": coords[site]
            
  }
  # define the clip tool
  clip = {
      "clip": {
          "aoi":clip_aoi
      }
  }
  # create an order request with the clipping tool
  request_clip = {
    "name": "site_" + str(site) + "_images_" + str(len(site_files[site])),
    "products": same_src_products,
    "tools": [clip]
  }
  # optional for zipping 
  #zip_delivery = {"delivery": {"single_archive": True, "archive_type": "zip"}}
  #request_clip.update(zip_delivery)
  clip_order_url = place_order(request_clip, auth)
  poll_for_success(clip_order_url, auth)
  r = requests.get(clip_order_url, auth=auth)
  response = r.json()
  results = response['_links']['results']
  fileLink = {}
  for result in results:
    if result['name'].endswith('.tif'):
      fileLink[result['name']] = result['location']
  # Download the tiff through link jpeg
  img_folder = '/content/drive/My Drive/CS_229_project/img_data/group2'
  for file in fileLink.keys():
      name = file
      im = requests.get(fileLink[file])
      tiff = Image.open(BytesIO(im.content))
      im = tiff.convert('RGB')
      im = im.resize((224, 224), Image.ANTIALIAS)
      name = str(name).rstrip(".tif") + '.jpg'
      name = str(site) + '_' + name.split('/')[-1]
      im.save(os.path.join(img_folder, name), 'JPEG')

<Response [202]>
a0b4d0db-751b-418c-9691-25683927372e
queued
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
success
<Response [202]>
e390ff8e-49da-4dda-b8b7-7d8ebf11c8c1
queued
running
running
running
running
running
running
running
running
running
running
success
<Response [202]>
aac3dd5a-4a45-4c43-960a-1588ebd992e9
queued
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
r

In [ ]:
r = requests.get('https://api.planet.com/compute/ops/orders/v2/', auth = auth)
response = r.json()
[response['orders'][i]['name'] for i in range(len(response['orders']))]
clip_order_url = response['orders'][3]['_links']["_self"]
clip_order_url

'https://api.planet.com/compute/ops/orders/v2/9f1b0152-0e37-464b-bcd5-b3d8c5d8b9ab'

In [ ]:
site = '73_1008'
r = requests.get(clip_order_url, auth=auth)
response = r.json()
results = response['_links']['results']
fileLink = {}
for result in results:
  if result['name'].endswith('.tif'):
    fileLink[result['name']] = result['location']
# Download the tiff through link jpeg
img_folder = '/content/drive/My Drive/CS_229_project/img_data/73_1008'
for file in fileLink.keys():
    name = file
    im = requests.get(fileLink[file])
    tiff = Image.open(BytesIO(im.content))
    im = tiff.convert('RGB')
    im = im.resize((224, 224), Image.ANTIALIAS)
    name = str(name).rstrip(".tif") + '.jpg'
    name = str(site) + '_' + name.split('/')[-1]
    im.save(os.path.join(img_folder, name), 'JPEG')